# Flojoy Cloud Python Client Example

To install the Flojoy Cloud Python client, simply do `pip install flojoy_cloud`.

In [ ]:
from flojoy_cloud import FlojoyCloud # Import the FlojoyCloud client

In [ ]:
from dotenv import load_dotenv
load_dotenv("./.env") # load environment variables from .env file

In [ ]:
import os

# Initialize the Python client
client = FlojoyCloud(workspace_secret=os.getenv("FLOJOY_CLOUD_WORKSPACE_SECRET"))

workspace_id = "workspace_k3w31luvuw85hnq5tx1nms70"

## Hardware Model

Every hardware device must have a hardware model. Therefore, before registering
your device to Flojoy Cloud, you must first create the corresponding model.

In this example, we are creating a "device model", which is meant for a 
standalone device that is not composed of anything else.

We also have support for a "system model", which is composed of multiple device 
models together, forming a larger system.

In [ ]:
device_model = client.create_device_model("HL9999", workspace_id)
print(device_model.name)

## Project

Nice! We just created a model. Now let's create a project to host everything we
do around this model.

A project contains a bunch of hardware instances of this model, and all 
the tests for this device model.

In [ ]:
project = client.create_project("HL9999 Testing", device_model.id, workspace_id)
print(project.name)

## Hardware Device

Let's register a hardware device instance to this project, so we can upload
some data for this particular device.

In [ ]:
hardware_device = client.create_device(workspace_id, "SN0001", device_model.id, project.id)
print(hardware_device.name)

## Test

This is the last setup step before we can start uploading data. 

We need to create a test such that we know which test is this data for.
It is a convenient way to group data into their respective test.

In [ ]:
# Create a test in the project we just created. 
test = client.create_test("Pass/Fail Test", project.id, "boolean")
print(test.name)

# This is the last setup step before we can upload some data!

## Data Uploading

As a quick example, let's say this test failed and we want to upload the status
data to Flojoy Cloud, we can pass a boolean to `upload`.


In [ ]:
status = False
client.upload(status, test.id, hardware_device.id, passed=True)

In [ ]:
measurements = client.get_all_measurements_by_test_id(test.id)
for m in measurements:
    print(m.data, m.hardware.name)

## Pulling Data Down

With our Python client, it is very convenient to pull data down into a Jupyter
notebook environment for example, to perform more data analysis.

In [ ]:
test_id = "test_dgxczo42p6ewud9fv242ltx0"
measurements = client.get_all_measurements_by_test_id(test_id)

In [ ]:
for m in measurements:
    print(m)

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()

for m in measurements:
    fig.add_trace(go.Scatter(x=m.data["value"]["x"], y=m.data["value"]["y"], name=m.hardware.name))

fig.show()